In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
# Reading the Test and Train Dataset Provided to Us

test =  pd.read_csv('/content/test_mSzZ8RL.csv') 
train = pd.read_csv( '/content/train_s3TEQDk.csv')

In [ ]:
# Separating the ID Columns from both the Test and Train Dataset

train_id=train['ID']
train=train.drop('ID', axis=1)
test_id=test['ID']
test=test.drop('ID', axis=1)

In [ ]:
#Setting the Null Values to a random value that will be later used in the Label encoder

train['Credit_Product'][pd.isnull(train['Credit_Product'])] = "other"
test['Credit_Product'][pd.isnull(test['Credit_Product'])] = "other"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
#Importing Label Encoder From SKLearn to encode the Strings value in to categorially encoded data

from sklearn.preprocessing  import LabelEncoder 

le = LabelEncoder()
var = ['Gender', 'Region_Code', 'Occupation', 'Channel_Code', 'Credit_Product', 'Is_Active']

for i in var:
  train[i] = le.fit_transform(train[i])
  test[i] = le.fit_transform(test[i])

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245725 entries, 0 to 245724
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   Gender               245725 non-null  int64
 1   Age                  245725 non-null  int64
 2   Region_Code          245725 non-null  int64
 3   Occupation           245725 non-null  int64
 4   Channel_Code         245725 non-null  int64
 5   Vintage              245725 non-null  int64
 6   Credit_Product       245725 non-null  int64
 7   Avg_Account_Balance  245725 non-null  int64
 8   Is_Active            245725 non-null  int64
 9   Is_Lead              245725 non-null  int64
dtypes: int64(10)
memory usage: 18.7 MB


In [ ]:
train.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,0,73,18,1,2,43,0,1045696,0,0
1,0,30,27,2,0,32,0,581988,0,0
2,0,56,18,3,2,26,0,1484315,1,0
3,1,34,20,2,0,19,0,470454,0,0
4,0,30,32,2,0,33,0,886787,0,0


In [ ]:
#Redistributing the Prediction Column from the Train dataset in X and Y train


ytrain=train['Is_Lead'].values
xtrain=train.drop(['Is_Lead'], axis=1)

In [ ]:
#forming a New Numpy array of the individual X and Y train


X = train.drop(['Is_Lead', 'Vintage', 'Avg_Account_Balance'], axis=1).values
y = train['Is_Lead'].values
X_test = test.drop(['Vintage', 'Avg_Account_Balance'], axis=1).values



In [ ]:

from sklearn.preprocessing import MaxAbsScaler

transformer = MaxAbsScaler().fit(X)
X = transformer.transform(X)
X_test = transformer.transform(X_test)

In [ ]:
#importing Stratified K fold and LightGBM for model Formation

from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
params = {'n_estimators': 20000, 'n_jobs': -1, 'random_state': 2, 'learning_rate': 0.012564209621859385, 'colsample_bytree': 0.48762749309989595}

model = lgb.LGBMClassifier(**params)


In [ ]:
np.unique(y)

array([0, 1])

In [ ]:
X_test

array([[1.        , 0.34117647, 0.11764706, ..., 0.        , 0.5       ,
        0.        ],
       [1.        , 0.50588235, 0.52941176, ..., 0.33333333, 1.        ,
        0.        ],
       [1.        , 0.36470588, 0.58823529, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.41176471, 0.11764706, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.62352941, 0.11764706, ..., 0.66666667, 0.        ,
        1.        ],
       [1.        , 0.31764706, 0.17647059, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
#Training the model

predictions = np.zeros(test.shape[0])

for i, (train_id, valid_id) in enumerate(skf.split(xtrain,ytrain)):
    print("fold ", i)  
    X_train, y_train = X[train_id], y[train_id]
    X_valid, y_valid = X[valid_id], y[valid_id]
    model.fit(X_train, y_train, eval_set =[(X_valid, y_valid)],  early_stopping_rounds=200, verbose=1000, eval_metric='auc')

    predictions += model.predict_proba(X_test)[:,1]

fold  0
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's auc: 0.866917	valid_0's binary_logloss: 0.350265
Early stopping, best iteration is:
[1301]	valid_0's auc: 0.867079	valid_0's binary_logloss: 0.349805
fold  1
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's auc: 0.867879	valid_0's binary_logloss: 0.350229
Early stopping, best iteration is:
[1088]	valid_0's auc: 0.867949	valid_0's binary_logloss: 0.350027
fold  2
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's auc: 0.864471	valid_0's binary_logloss: 0.354222
[2000]	valid_0's auc: 0.864833	valid_0's binary_logloss: 0.353551
Early stopping, best iteration is:
[2102]	valid_0's auc: 0.864846	valid_0's binary_logloss: 0.353529
fold  3
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's auc: 0.862879	valid_0's binary_logloss: 0.354019
Early stopping, best iteration is:
[1098]	valid_0's auc: 0.86296	valid_0's binary_l

In [ ]:
#final prediction value is calculated 

finals = predictions/5

In [ ]:
lgb_pred = model.predict(test)

In [ ]:
#Forming the final submission CSV File


df = pd.DataFrame()
df['ID']=test_id
df['Is_Lead']=finals
df.to_csv('file_name.csv', index=False)